crf测试pku中文语料第1版

训练集：train_utf16.ner的随机70%

测试集：train_utf16.ner的随机30%

提取特征：word,label,istitle(),isdigit()

结果：precision/recall全是1

In [167]:
import random
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import pycrfsuite

In [168]:
if __name__=='__main__':
    train_sents = []
    test_sents = []
    f = open('./train_utf16.ner', 'r', encoding='utf_16_le')
    pairList = []
    for line in f:
        line = line.encode('utf-8').decode('utf-8-sig').strip()  # 去掉首字前的\ufeff
        if line:
            l = line.split(' ')
            pairList.append([l[0], l[1]])
        else:
            r = random.random()
            if r < 0.7:
                train_sents.append(pairList)
            else:
                test_sents.append(pairList)
            pairList = []
    f.close()

In [169]:
print(train_sents[0])

[['精', 'N'], ['品', 'N'], ['、', 'N'], ['专', 'N'], ['题', 'N'], ['、', 'N'], ['系', 'N'], ['列', 'N'], ['、', 'N'], ['稀', 'N'], ['见', 'N'], ['程', 'N'], ['度', 'N'], ['才', 'N'], ['是', 'N'], ['质', 'N'], ['量', 'N'], ['的', 'N'], ['核', 'N'], ['心', 'N'], ['。', 'N']]


In [170]:
def word2features(sent, i):  # for i in range(len(sent))
    word = sent[i][0]
    label = sent[i][1]
    features = [  # 每个当前字的features
        'bias',
        'word=' + word,
#         'word[-3:]=' + word[-3:],
#         'word[-2:]=' + word[-2:],
#         'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'label=' + label,
#         'label[:2]=' + label[:2],
    ]
    if i > 0:
        word1 = sent[i - 1][0]
        label1 = sent[i - 1][1]
        features.extend([  # 前一个单词的features
            '-1:word=' + word1,
            '-1:word.istitle=%s' % word1.istitle(),
#             '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + label1,
#             '-1:label[:2]=' + label1[:2],
        ])
    else:
        features.append('BOS')
    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        label1 = sent[i + 1][1]
        features.extend([  # 后一个单词的features
            '+1:word=' + word1,
            '+1:word.istitle=%s' % word1.istitle(),
#             '+1:word.isupper=%s' % word1.isupper(),
            '+1:label=' + label1,
#             '+1:label[:2]=' + label1[:2],
        ])
    else:
        features.append('EOS')
    return features

In [171]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    return [label for token, label in sent]


def sent2tokens(sent):
    return [token for token, label in sent]


In [172]:
print(sent2features(train_sents[0])[0])  # 第一个list的第一个位置上的word,按照自己定义的特征提取出一些信息

['bias', 'word=精', 'word.istitle=False', 'word.isdigit=False', 'label=N', 'BOS', '+1:word=品', '+1:word.istitle=False', '+1:label=N']


In [173]:
print(sent2labels(train_sents[0])[0])  # 第一个list的第一个位置上的label

N


In [174]:
print(sent2features(train_sents[0])[-1])  # 第一个list的第一个位置上的word,按照自己定义的特征提取出一些信息

['bias', 'word=。', 'word.istitle=False', 'word.isdigit=False', 'label=N', '-1:word=心', '-1:word.istitle=False', '-1:postag=N', 'EOS']


In [175]:
X_train = [sent2features(s) for s in train_sents]  # 不加[]是generator类型，不会输出。加了[]，可直接输出加了[]之后的结果。
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]  # 格式 [[[每个word的特征],[每个word的特征]],（到此处是一个s）,[(每个word的特征)],...]，
y_test = [sent2labels(s) for s in test_sents] 

In [176]:
trainer = pycrfsuite.Trainer(verbose=False)

In [177]:
for xseq, yseq in zip(X_train, y_train):  # zip,打包为元组的列表
    trainer.append(xseq, yseq)  # 要train的data

In [178]:
trainer.set_params({
    'c1': 1.0,
    'c2': 1e-3,
    'max_iterations': 50,
    'feature.possible_transitions': True
})

In [179]:
print('trainer.params():', trainer.params())

trainer.params(): ['feature.minfreq', 'feature.possible_states', 'feature.possible_transitions', 'c1', 'c2', 'max_iterations', 'num_memories', 'epsilon', 'period', 'delta', 'linesearch', 'max_linesearch']


In [180]:
trainer.train('conll2002_1.crfsuite')

In [181]:
print('trainer.logparser.last_iteration:', trainer.logparser.last_iteration)

trainer.logparser.last_iteration: {'num': 50, 'scores': {}, 'loss': 89.900078, 'feature_norm': 29.618714, 'error_norm': 0.053167, 'active_features': 13, 'linesearch_trials': 1, 'linesearch_step': 1.0, 'time': 0.33}


In [182]:
print(len(trainer.logparser.iterations), trainer.logparser.iterations[-1])

50 {'num': 50, 'scores': {}, 'loss': 89.900078, 'feature_norm': 29.618714, 'error_norm': 0.053167, 'active_features': 13, 'linesearch_trials': 1, 'linesearch_step': 1.0, 'time': 0.33}


In [183]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002_1.crfsuite')  # 使用训练后的模型，创建用于测试的标注器。

In [184]:
example_sent = test_sents[1]
print('example_sent:', example_sent)
print("token of example_sent:", ' '.join(sent2tokens(example_sent)), end='\n\n')

example_sent: [['以', 'N'], ['家', 'N'], ['乡', 'N'], ['的', 'N'], ['历', 'N'], ['史', 'N'], ['文', 'N'], ['献', 'N'], ['、', 'N'], ['特', 'N'], ['定', 'N'], ['历', 'N'], ['史', 'N'], ['时', 'N'], ['期', 'N'], ['书', 'N'], ['刊', 'N'], ['、', 'N'], ['某', 'N'], ['一', 'N'], ['名', 'N'], ['家', 'N'], ['或', 'N'], ['名', 'N'], ['著', 'N'], ['的', 'N'], ['多', 'N'], ['种', 'N'], ['出', 'N'], ['版', 'N'], ['物', 'N'], ['为', 'N'], ['专', 'N'], ['题', 'N'], ['，', 'N'], ['注', 'N'], ['意', 'N'], ['精', 'N'], ['品', 'N'], ['、', 'N'], ['非', 'N'], ['卖', 'N'], ['品', 'N'], ['、', 'N'], ['纪', 'N'], ['念', 'N'], ['品', 'N'], ['，', 'N'], ['集', 'N'], ['成', 'N'], ['系', 'N'], ['列', 'N'], ['，', 'N'], ['那', 'N'], ['收', 'N'], ['藏', 'N'], ['的', 'N'], ['过', 'N'], ['程', 'N'], ['就', 'N'], ['已', 'N'], ['经', 'N'], ['够', 'N'], ['您', 'N'], ['玩', 'N'], ['味', 'N'], ['无', 'N'], ['穷', 'N'], ['了', 'N'], ['。', 'N']]
token of example_sent: 以 家 乡 的 历 史 文 献 、 特 定 历 史 时 期 书 刊 、 某 一 名 家 或 名 著 的 多 种 出 版 物 为 专 题 ， 注 意 精 品 、 非 卖 品 、 纪 念 品 ， 集 成 系 列 ， 那 收 藏 的 过 程 就 已 

In [185]:
print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

Predicted: N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N
Correct:   N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N


In [186]:
def bio_classification_report(y_true, y_pred):
    '''
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    '''
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
    tagset = set(lb.classes_) - {'N'}  # 从9个labels中去除大写字母N
    print(tagset)  # {'I-LOC', 'B-MISC', 'B-ORG', 'I-ORG', 'B-LOC', 'I-MISC', 'I-PER', 'B-PER'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    print(tagset)  # ['B-LOC', 'I-LOC', 'B-MISC', 'I-MISC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER']
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}  # idx:0,1,2,3,4,5,6,7; cls:8个labels;

    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels=[class_indices[cls] for cls in tagset],
        target_names=tagset,
    )

In [194]:
y_pred = [tagger.tag(xseq) for xseq in X_test]

print(bio_classification_report(y_test, y_pred))

{'B-PER', 'I-LOC', 'B-ORG', 'I-ORG', 'I-PER', 'B-LOC'}
['B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER']
             precision    recall  f1-score   support

      B-LOC       1.00      1.00      1.00      5374
      I-LOC       1.00      1.00      1.00      7107
      B-ORG       1.00      1.00      1.00      3085
      I-ORG       1.00      1.00      1.00     12315
      B-PER       1.00      1.00      1.00      2707
      I-PER       1.00      1.00      1.00      5235

avg / total       1.00      1.00      1.00     35823



In [192]:
print(test_sents[0])

[['藏', 'N'], ['书', 'N'], ['本', 'N'], ['来', 'N'], ['就', 'N'], ['是', 'N'], ['所', 'N'], ['有', 'N'], ['传', 'N'], ['统', 'N'], ['收', 'N'], ['藏', 'N'], ['门', 'N'], ['类', 'N'], ['中', 'N'], ['的', 'N'], ['第', 'N'], ['一', 'N'], ['大', 'N'], ['户', 'N'], ['，', 'N'], ['只', 'N'], ['是', 'N'], ['我', 'N'], ['们', 'N'], ['结', 'N'], ['束', 'N'], ['温', 'N'], ['饱', 'N'], ['的', 'N'], ['时', 'N'], ['间', 'N'], ['太', 'N'], ['短', 'N'], ['而', 'N'], ['已', 'N'], ['。', 'N']]


In [190]:
print(y_test[0])

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']


In [189]:
print(y_pred[0])

['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']


In [193]:
label_B_LOC = 0
label_I_LOC = 0
label_B_ORG = 0
label_I_ORG = 0
label_B_PER = 0
label_I_PER = 0
label_N = 0        
index = 0

for sent in y_pred:
    if sent:
        for label in sent:
            index += 1
            if label == 'B-LOC':
                label_B_LOC += 1
            if label == 'I-LOC':
                label_I_LOC += 1
            if label == 'B-ORG':
                label_B_ORG += 1
            if label == 'I-ORG':
                label_I_ORG += 1
            if label == 'B-PER':
                label_B_PER += 1
            if label == 'I-PER':
                label_I_PER += 1
            if label == 'N':
                label_N += 1

print('label是B_LOC的单词共:', label_B_LOC)
print('label是I_LOC的单词共:', label_I_LOC)
print('label是B_ORG的单词共:', label_B_ORG)
print('label是I_ORG的单词共:', label_I_ORG)
print('label是B_PER的单词共:', label_B_PER)
print('label是I_PER的单词共:', label_I_PER)
print('除去标签是N的单词共:', label_B_LOC+label_B_ORG+label_B_PER+label_I_LOC+label_I_ORG+label_I_PER)
print('label是N的单词共:', label_N)
print('单词总共:',label_B_LOC+label_B_ORG+label_B_PER+label_I_LOC+label_I_ORG+label_I_PER+label_N)
print(index)

label是B_LOC的单词共: 5374
label是I_LOC的单词共: 7107
label是B_ORG的单词共: 3085
label是I_ORG的单词共: 12315
label是B_PER的单词共: 2707
label是I_PER的单词共: 5235
除去标签是N的单词共: 35823
label是N的单词共: 288971
单词总共: 324794
324794
